In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def texto_a_fecha(texto):
    hoy = datetime.now()
    palabras = texto.split()
    
    if len(palabras) < 2:
        return hoy.strftime('%m/%d/%Y')
    
    cantidad = palabras[1]
    unidad = palabras[2]
    
    if cantidad == 'un' or cantidad == 'una':
        cantidad = 1
    else:
        try:
            cantidad = int(cantidad)
        except ValueError:
            return hoy.strftime('%m/%d/%Y')
    
    if 'año' in unidad:
        return (hoy - timedelta(days=cantidad*365)).strftime('%m/%d/%Y')
    elif 'mes' in unidad:
        return (hoy - timedelta(days=cantidad*30)).strftime('%m/%d/%Y')
    elif 'semana' in unidad:
        return (hoy - timedelta(weeks=cantidad)).strftime('%m/%d/%Y')
    else:
        return hoy.strftime('%m/%d/%Y')

In [3]:
name = 'bares'
df = pd.read_csv(f'comentarios_{name}.xls')
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
df['Fecha_numero'] = df['Fecha'].apply(texto_a_fecha)
df.to_csv(f'comentarios_{name}.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'comentarios_bares.xls'

In [6]:
list(df['Nombre'].unique())

['SINNERS Microcervecería Floresta',
 'La Santa Cantina Pop',
 'Holy Krank Pub',
 'Abysmo GastroPub UIO',
 'Zmus Bar',
 "Finn McCool's",
 nan,
 'Indie Beer Company',
 'Lucky Charm Pub & Beer',
 'Rock & Beef Resto-Bar',
 'Odeon Pub House',
 'The Boot Sports Bar',
 'Duff Resto Bar',
 'Tijuana']

# Mapa interactivo

In [21]:
import folium
import pandas as pd
from folium.plugins import MiniMap
import math
df = pd.read_csv(f'comentarios_bares.csv')

In [24]:
def convertir_coordenadas(arreglo):

    coordenadas = []
    for coord_str in arreglo:
        # Eliminar paréntesis y comillas
        coord_str = coord_str.replace("(", "").replace(")", "").replace("'", "")
        # Separar la latitud y longitud
        lat_str, lon_str = coord_str.split(", ")
        # Convertir a floats
        lat = float(lat_str)
        lon = float(lon_str)
        # Añadir a la lista de coordenadas
        coordenadas.append((lat, lon))
    return coordenadas

In [22]:
def calcular_punto_medio(coordenadas):

    if len(coordenadas) == 0:
        return None
    
    x = 0
    y = 0
    z = 0

    for lat, lon in coordenadas:
        lat_rad = math.radians(lat)
        lon_rad = math.radians(lon)
        
        x += math.cos(lat_rad) * math.cos(lon_rad)
        y += math.cos(lat_rad) * math.sin(lon_rad)
        z += math.sin(lat_rad)
    
    total = len(coordenadas)
    x /= total
    y /= total
    z /= total
    
    lon_media = math.atan2(y, x)
    hyp = math.sqrt(x * x + y * y)
    lat_media = math.atan2(z, hyp)
    
    lat_media = math.degrees(lat_media)
    lon_media = math.degrees(lon_media)
    
    return (lat_media, lon_media)

In [6]:
def extraer_info_empresa(df, nombre):
    df = df.pivot_table(index='Nombre', values=['Latitud', 'Longitud', 'Dirección', 'Tipo Establecimiento'], aggfunc='first')
    df = df.loc[f'{nombre}',['Latitud', 'Longitud', 'Dirección', 'Tipo Establecimiento']]
    return [[df['Latitud'], df['Longitud']], df['Dirección'], df['Tipo Establecimiento']]

In [29]:
df['Nombre'].unique()

array(['SINNERS Microcervecería Floresta', 'La Santa Cantina Pop',
       'Holy Krank Pub', 'Abysmo GastroPub UIO', 'Zmus Bar',
       "Finn McCool's", nan, 'Indie Beer Company',
       'Lucky Charm Pub & Beer', 'Rock & Beef Resto-Bar',
       'Odeon Pub House', 'The Boot Sports Bar', 'Duff Resto Bar',
       'Tijuana'], dtype=object)

In [28]:
def mapa_floresta(df, nombre, total = False):
    if total == False:
        coord, dir, tipo = extraer_info_empresa(df, nombre)
        popuptext = f'''
        <div style="text-align: center; width: 200px;">
            <b>{tipo}</b><br>
            <b>{nombre}</b><br>
            <b>({coord[0]}, {coord[1]})</b><br>
            <b>{dir}</b>
        </div>
        '''
        floresta = folium.Map(location = coord, zoom_start=18)
        folium.Marker(location=coord, popup=folium.Popup(popuptext, max_width=300)).add_to(floresta)
        folium.Circle(location= coord, color = 'purple', fill_color = 'red', radius = 10, weight = 4, fill_opacity = 0.5).add_to(floresta)
        return floresta
    try:
        coordenadas = convertir_coordenadas(df['Coordenadas'].unique())
        punto_medio = calcular_punto_medio(coordenadas)
        floresta = folium.Map(location = punto_medio, zoom_start=16)
        for nombres in df['Nombre'].dropna().unique():
            coord, dir, tipo = extraer_info_empresa(df, nombres)
            popuptext = f'''
            <div style="text-align: center; width: 200px;">
                <b>{tipo}</b><br>
                <b>{nombres}</b><br>
                <b>({coord[0]}, {coord[1]})</b><br>
                <b>{dir}</b>
            </div>
            '''
            
            if nombres == nombre:
                folium.Circle(location= coord, color = 'red', fill_color = 'red', radius = 12, weight = 5, fill_opacity = 0.5).add_to(floresta)
                folium.Marker(location=coord, popup=folium.Popup(popuptext, max_width=300), icon=folium.Icon(color = 'red')).add_to(floresta)
            else:
                folium.Circle(location= coord, color = 'blue', fill_color = 'blue', radius = 10, weight = 4, fill_opacity = 0.5).add_to(floresta)
                folium.Marker(location=coord, popup=folium.Popup(popuptext, max_width=300), icon=folium.Icon(color = 'blue')).add_to(floresta)
        return floresta
    except:
        return floresta
    

mapa_floresta(df, 'Tijuana', True)

# Unir dataframes comentarios

In [1]:
import pandas as pd
bares = pd.read_csv('comentarios_bares.csv')
hoteles = pd.read_csv('comentarios_hoteles.csv')
restaurantes = pd.read_csv('comentarios_restaurante.csv')

In [3]:
pd.merge(bares,hoteles)

,Comentario,Fecha,Estrellas,Nombre,Tipo Establecimiento,Tipo,Dirección,Coordenadas,Latitud,Longitud,Calificación,Fecha_numero


In [7]:
df_comentarios = pd.concat([bares,hoteles,restaurantes], axis=0)

In [9]:
df_comentarios.to_csv('comenterios.csv')

# Series de tiempo

In [5]:
from datetime import datetime, timedelta
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import folium
import math

In [16]:
def texto_a_fecha(texto):
    hoy = datetime.now()
    palabras = texto.split()
    
    if len(palabras) < 2:
        return hoy.strftime('%m/%d/%Y')
    
    cantidad = palabras[1]
    unidad = palabras[2]
    
    if cantidad == 'un' or cantidad == 'una':
        cantidad = 1
    else:
        try:
            cantidad = int(cantidad)
        except ValueError:
            return hoy.strftime('%m/%d/%Y')
    
    if 'año' in unidad:
        return (hoy - timedelta(days=cantidad*365)).strftime('%m/%d/%Y')
    elif 'mes' in unidad:
        return (hoy - timedelta(days=cantidad*30)).strftime('%m/%d/%Y')
    elif 'semana' in unidad:
        return (hoy - timedelta(weeks=cantidad)).strftime('%m/%d/%Y')
    else:
        return hoy.strftime('%m/%d/%Y')
    

def dataframe_series_tiempo(df):
    df['Fecha_numero'] = pd.to_datetime(df['Fecha_numero'])
    df['Año-Mes'] = df['Fecha_numero'].dt.to_period('M')
    df_conteo = df.groupby(['Nombre', 'Año-Mes', 'Calificación']).size().unstack(fill_value=0)
    df_conteo.columns = ['Malos', 'Neutros', 'Buenos']
    df_conteo = df_conteo.reset_index()
    df_conteo['Fecha'] = df_conteo['Año-Mes'].dt.to_timestamp()
    df_conteo = df_conteo.drop('Año-Mes', axis=1)
    return df_conteo


def serie_tiempo_empresa(df, nombre_empresa='Total'):
    df = dataframe_series_tiempo(df)
    if nombre_empresa != 'Total':
        # Filtrar el DataFrame para la empresa específica
        df = df[df['Nombre'] == nombre_empresa]
    # Establecer la fecha como índice
    df = df.set_index('Fecha')
    # Seleccionar solo las columnas de interés
    return df[['Buenos', 'Neutros', 'Malos']]
     


def graficar_serie_tiempo(df, nombre_empresa='Total'):
    series = serie_tiempo_empresa(df, nombre_empresa)
    fig = go.Figure()

    # Añadir las series de tiempo al gráfico
    for column in series.columns:
        fig.add_trace(go.Scatter(x=series.index, y=series[column], mode='lines', name=column))
    if nombre_empresa=='Total': 
        nombre_empresa = 'todas las empresas'
    # Configurar el layout del gráfico
    fig.update_layout(
        title=f"Serie de tiempo de comentarios para {nombre_empresa}",
        xaxis_title='Fecha',
        yaxis_title='Número de comentarios',
        template='plotly',
        hovermode='x unified'
    )

In [13]:
df = pd.read_csv('comentarios.csv')

In [18]:
serie_tiempo_empresa(df, 'Total')

,Buenos,Neutros,Malos
Fecha,,,
2022-06-01,1,0,0
2023-08-01,1,0,0
2023-09-01,2,0,0
2017-06-01,17,1,1
2018-06-01,51,2,2
...,...,...,...
2024-01-01,1,0,0
2024-02-01,1,1,0
2024-03-01,1,0,0
